In [1]:
import os
from glob import glob
from PIL import Image

import numpy
import coremltools

import keras
from keras.models import *
from keras.layers import *
from keras.callbacks import *

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

tf.estimator package not installed.
tf.estimator package not installed.


In [2]:
def create_model(image_shape, class_count):
    model = Sequential()

    model.add(Conv2D(32, (5, 5), input_shape=(image_shape[0], image_shape[1], 1))) # First convolution Layer
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3))) # Second Convolution Layer
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(256)) # FC layer with 256 neurons
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(class_count)) # Result layer
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy']) 
    
    return model

In [3]:
def load_image(path, image_shape):
    img = Image.open(path).convert('LA').resize(image_shape, Image.ANTIALIAS)
    pixels = [f[0] for f in list(img.getdata())] # take first channel form each pixel
    return pixels

In [4]:
def load_dataset(name, image_shape):
    dataset = []
    classes = []
    buckets = [
        './data/',
    ]
    for bucket in buckets:
        path = bucket + name + '/'
        if not os.path.isdir(path):
            continue
        for cls in os.listdir(path):
            cls_path = os.path.join(path, cls)
            if not os.path.isdir(cls_path):
                continue
            for item in glob(os.path.join(cls_path, '*.png')):
                dataset.append(load_image(item, image_shape))
                classes.append(cls)
    return numpy.array(dataset), numpy.array(classes)

In [5]:
def train_model(name, image_shape):
    dataset, classes = load_dataset(name, image_shape)
    dataset_train, dataset_test, classes_train, classes_test \
        = train_test_split(dataset, classes, test_size=0.25, random_state=42)
    dataset_train = dataset_train.reshape(dataset_train.shape[0], 32, 32, 1).astype('float32') / 255
    dataset_test = dataset_test.reshape(dataset_test.shape[0], 32, 32, 1).astype('float32') / 255
    
    encoder = LabelBinarizer()
    encoder.fit(classes)
    classes_train = encoder.transform(classes_train)
    classes_test = encoder.transform(classes_test)
    
    model = create_model(image_shape, len(encoder.classes_))
    history = model.fit(
        dataset_train, classes_train,
        epochs=25, batch_size=128, callbacks=[],
        validation_data=(dataset_test, classes_test))

    return model, map(lambda x: str(x), encoder.classes_)

In [6]:
def convert_model(model, name, classes):
    coreml_model = coremltools.converters.keras.convert(
        model, # name + '.h5',
        input_names='image',
        image_input_names='image',
        output_names='output',
        class_labels=classes,
        image_scale=1/255.)
    coreml_model.save("./models/" + name + '.mlmodel')

In [7]:
def build_model(name, output_name, image_shape):
    model, classes = train_model(name, image_shape)
    convert_model(model, output_name, classes)

In [8]:
build_model('hp', 'HpClassifier', (32, 32))

Train on 3489 samples, validate on 1164 samples
Epoch 1/25
3489/3489 [==============================] - 3s 750us/step - loss: 1.5589 - acc: 0.5277 - val_loss: 0.1805 - val_acc: 0.9966
Epoch 2/25
3489/3489 [==============================] - 3s 728us/step - loss: 0.1269 - acc: 0.9748 - val_loss: 0.0312 - val_acc: 0.9897
Epoch 3/25
3489/3489 [==============================] - 3s 739us/step - loss: 0.0357 - acc: 0.9900 - val_loss: 0.0412 - val_acc: 0.9845
Epoch 4/25
3489/3489 [==============================] - 3s 724us/step - loss: 0.0109 - acc: 0.9974 - val_loss: 0.0012 - val_acc: 1.0000
Epoch 5/25
3489/3489 [==============================] - 2s 674us/step - loss: 0.0151 - acc: 0.9957 - val_loss: 0.0014 - val_acc: 1.0000
Epoch 6/25
3489/3489 [==============================] - 3s 738us/step - loss: 0.0036 - acc: 0.9991 - val_loss: 4.2226e-04 - val_acc: 1.0000
Epoch 7/25
3489/3489 [==============================] - 2s 683us/step - loss: 0.0069 - acc: 0.9983 - val_loss: 8.4040e-04 - val_acc: